In [13]:
import pandas as pd
import os
from rich.console import Console
from rich.table import Table
from skopt.space import Real, Integer, Categorical
from sklearn.model_selection import KFold

In [14]:
os.chdir(r'C:\SML_Projects\SML_CVE_type_cwe_predict')
os.makedirs('results', exist_ok=True)

In [15]:
import warnings
warnings.filterwarnings("ignore")

In [16]:
x_train = pd.read_csv('data/preprocessed/preprocessed_x_train.csv')
x_test = pd.read_csv('data/preprocessed/preprocessed_x_test.csv')

y_train = pd.read_csv('data/split/y_train.csv')[['type', 'cvss_score']]
y_test  = pd.read_csv('data/split/y_test.csv')[['type', 'cvss_score']]

In [17]:
kf = KFold(n_splits=3, shuffle=True, random_state=42)

In [18]:
import optuna
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier,
    HistGradientBoostingClassifier, AdaBoostClassifier,
    BaggingClassifier, VotingClassifier, StackingClassifier
)

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [19]:
base_estimators = [
    ('rf', RandomForestClassifier(random_state=42)),
    ('xgb', XGBClassifier(random_state=42, eval_metric='logloss')),
    ('lgbm', LGBMClassifier(random_state=42))
]

In [20]:
def get_model(trial, name):

    # ===== Linear =====
    if name == 'Logistic Regression':
        return LogisticRegression(
            C=trial.suggest_float('C', 1e-2, 10.0, log=True),
            solver=trial.suggest_categorical('solver', ['lbfgs', 'liblinear']),
            max_iter=1000
        )

    # ===== Tree =====
    if name == 'DecisionTree':
        return DecisionTreeClassifier(
            max_depth=trial.suggest_int('max_depth', 2, 30),
            min_samples_split=trial.suggest_int('min_samples_split', 2, 20),
            min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 10),
            random_state=42
        )

    if name == 'RandomForest':
        return RandomForestClassifier(
            n_estimators=trial.suggest_int('n_estimators', 100, 600),
            max_depth=trial.suggest_int('max_depth', 3, 30),
            min_samples_split=trial.suggest_int('min_samples_split', 2, 10),
            min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 4),
            max_features=trial.suggest_categorical('max_features', ['sqrt', 'log2']),
            n_jobs=-1,
            random_state=42
        )

    if name == 'ExtraTrees':
        return ExtraTreesClassifier(
            n_estimators=trial.suggest_int('n_estimators', 100, 600),
            max_depth=trial.suggest_int('max_depth', 3, 30),
            min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 5),
            n_jobs=-1,
            random_state=42
        )

    if name == 'Bagging':
        return BaggingClassifier(
            n_estimators=trial.suggest_int('n_estimators', 50, 400),
            max_samples=trial.suggest_float('max_samples', 0.5, 1.0),
            max_features=trial.suggest_float('max_features', 0.5, 1.0),
            random_state=42
        )

    if name == 'Bagged DT':
        return BaggingClassifier(
            estimator=DecisionTreeClassifier(),
            n_estimators=trial.suggest_int('n_estimators', 50, 400),
            random_state=42
        )

    # ===== Boosting =====
    if name == 'GradientBoosting':
        return GradientBoostingClassifier(
            n_estimators=trial.suggest_int('n_estimators', 100, 500),
            learning_rate=trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
            max_depth=trial.suggest_int('max_depth', 2, 6),
            random_state=42
        )

    if name == 'HistGradientBoosting':
        return HistGradientBoostingClassifier(
            learning_rate=trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
            max_depth=trial.suggest_int('max_depth', 2, 10),
            random_state=42
        )

    if name == 'AdaBoost':
        return AdaBoostClassifier(
            n_estimators=trial.suggest_int('n_estimators', 100, 500),
            learning_rate=trial.suggest_float('learning_rate', 0.01, 1.0, log=True),
            random_state=42
        )

    if name == 'XGBoost':
        return XGBClassifier(
            n_estimators=trial.suggest_int('n_estimators', 100, 500),
            max_depth=trial.suggest_int('max_depth', 3, 10),
            learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            subsample=trial.suggest_float('subsample', 0.6, 1.0),
            colsample_bytree=trial.suggest_float('colsample_bytree', 0.6, 1.0),
            eval_metric='logloss',
            random_state=42
        )

    if name == 'LGBMClassifier':
        return LGBMClassifier(
            n_estimators=trial.suggest_int('n_estimators', 100, 500),
            learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            num_leaves=trial.suggest_int('num_leaves', 31, 255),
            random_state=42
        )

    # ===== Ensembles =====
    if name == 'Hard Voting':
        return VotingClassifier(estimators=base_estimators, voting='hard')

    if name == 'Soft Voting':
        return VotingClassifier(estimators=base_estimators, voting='soft')

    if name == 'Stacking':
        return StackingClassifier(
            estimators=base_estimators,
            final_estimator=LogisticRegression(max_iter=1000)
        )

In [21]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score

def objective(trial, name):

    base_model = get_model(trial, name)
    model = MultiOutputClassifier(base_model)

    y_pred = cross_val_predict(
        model,
        x_train,
        y_train,
        cv=kf,
        n_jobs=-1
    )

    recall_type = recall_score(
        y_train['type'],
        y_pred[:, 0],
        average='macro'
    )

    recall_cvss = recall_score(
        y_train['cvss_score'],
        y_pred[:, 1],
        average='macro'
    )

    return (recall_type + recall_cvss) / 2

In [ ]:
models = [
    'Logistic Regression', 'DecisionTree', 'RandomForest', 'ExtraTrees', 'Bagging', 
    'Bagged DT', 'GradientBoosting', 'HistGradientBoosting', 'AdaBoost', 'XGBoost',
    'LGBMClassifier', 'Hard Voting', 'Soft Voting', 'Stacking'
]

results = []

for name in models:
    print("\n" + "=" * 60)
    print(f"🔥 NOW TUNING: {name}")
    print("=" * 60)

    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, name), n_trials=30)

    best_base = get_model(study.best_trial, name)
    best_model = MultiOutputClassifier(best_base)

    best_model.fit(x_train, y_train)
    y_pred = best_model.predict(x_test)


    # ===== TYPE =====
    acc_type  = accuracy_score(y_test['type'], y_pred[:,0])
    prec_type = precision_score(y_test['type'], y_pred[:,0], average='macro')
    rec_type  = recall_score(y_test['type'], y_pred[:,0], average='macro')
    f1_type   = f1_score(y_test['type'], y_pred[:,0], average='macro')

    type_scores = cross_val_score(best_base, x_train, y_train['type'], cv=kf, scoring='f1_macro', n_jobs=-1)
    type_kf_mean = type_scores.mean()
    type_kf_std = type_scores.std()

    # ===== CVSS =====
    acc_cvss  = accuracy_score(y_test['cvss_score'], y_pred[:,1])
    prec_cvss = precision_score(y_test['cvss_score'], y_pred[:,1], average='macro')
    rec_cvss  = recall_score(y_test['cvss_score'], y_pred[:,1], average='macro')
    f1_cvss   = f1_score(y_test['cvss_score'], y_pred[:,1], average='macro')

    cvss_scores = cross_val_score(best_base, x_train, y_train['cvss_score'], cv=kf, scoring='f1_macro', n_jobs=-1)
    cvss_kf_mean = cvss_scores.mean()
    cvss_kf_std = cvss_scores.std()

    # ===== MEAN =====
    f1_mean = (f1_type + f1_cvss) / 2

    # ===== PRINT RESULTS =====
    print(f"\n📊 RESULTS FOR MODEL: {name}")
    print("=" * 55)

    # ===================== TYPE =====================
    print("🎯🎯 TARGET: TYPE")
    print("-" * 55)
    print(f"📈 Accuracy        : {acc_type:.2f}")
    print(f"🧠 Precision (mac) : {prec_type:.2f}")
    print(f"🔁 Recall    (mac) : {rec_type:.2f}")
    print(f"🏆 F1-score  (mac) : {f1_type:.2f}")
    print(f"📊 K-Fold F1 Mean  : {type_kf_mean:.2f}")
    print(f"📉 K-Fold F1 Std   : {type_kf_std:.2f}")

    print("\n" + "-" * 55)

    # ===================== CVSS =====================
    print("🎯 TARGET: CVSS_SCORE")
    print("-" * 55)
    print(f"📈 Accuracy        : {acc_cvss:.2f}")
    print(f"🧠 Precision (mac) : {prec_cvss:.2f}")
    print(f"🔁 Recall    (mac) : {rec_cvss:.2f}")
    print(f"🏆 F1-score  (mac) : {f1_cvss:.2f}")
    print(f"📊 K-Fold F1 Mean  : {cvss_kf_mean:.2f}")
    print(f"📉 K-Fold F1 Std   : {cvss_kf_std:.2f}")

    print("\n" + "=" * 55)
    print(f"📌 MEAN F1 (TYPE + CVSS) / 2 : {f1_mean:.2f}")
    print("=" * 55)


    results.append([
        name,

        # ===== TYPE =====
        acc_type,
        prec_type,
        rec_type,
        f1_type,
        type_kf_mean,
        type_kf_std,

        # ===== CVSS =====
        acc_cvss,
        prec_cvss,
        rec_cvss,
        f1_cvss,
        cvss_kf_mean,
        cvss_kf_std
    ])

[I 2026-01-17 11:03:46,750] A new study created in memory with name: no-name-d86f4222-d6b0-410c-bdd6-e73c34bd88b7



🔥 NOW TUNING: Logistic Regression


[I 2026-01-17 11:03:49,719] Trial 0 finished with value: 0.5214358858346004 and parameters: {'C': 0.10538748974652203, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.5214358858346004.
[I 2026-01-17 11:03:52,401] Trial 1 finished with value: 0.5235182854201758 and parameters: {'C': 0.3286854872303982, 'solver': 'lbfgs'}. Best is trial 1 with value: 0.5235182854201758.
[I 2026-01-17 11:04:00,537] Trial 2 finished with value: 0.5182944723658953 and parameters: {'C': 0.22815696726795595, 'solver': 'liblinear'}. Best is trial 1 with value: 0.5235182854201758.
[I 2026-01-17 11:04:10,398] Trial 3 finished with value: 0.5185208083272399 and parameters: {'C': 1.3040019144118806, 'solver': 'liblinear'}. Best is trial 1 with value: 0.5235182854201758.
[I 2026-01-17 11:04:18,123] Trial 4 finished with value: 0.5181019411733838 and parameters: {'C': 0.10436177280780859, 'solver': 'liblinear'}. Best is trial 1 with value: 0.5235182854201758.
[I 2026-01-17 11:04:20,384] Trial 5 finished with value


📊 RESULTS FOR MODEL: Logistic Regression
🎯🎯 TARGET: TYPE
-------------------------------------------------------
📈 Accuracy        : 0.78
🧠 Precision (mac) : 0.79
🔁 Recall    (mac) : 0.72
🏆 F1-score  (mac) : 0.74
📊 K-Fold F1 Mean  : 0.73
📉 K-Fold F1 Std   : 0.00

-------------------------------------------------------
🎯 TARGET: CVSS_SCORE
-------------------------------------------------------
📈 Accuracy        : 0.57
🧠 Precision (mac) : 0.48
🔁 Recall    (mac) : 0.30
🏆 F1-score  (mac) : 0.27
📊 K-Fold F1 Mean  : 0.33
📉 K-Fold F1 Std   : 0.00

📌 MEAN F1 (TYPE + CVSS) / 2 : 0.50

🔥 NOW TUNING: DecisionTree


[I 2026-01-17 11:05:42,338] Trial 0 finished with value: 0.6936605538806662 and parameters: {'max_depth': 12, 'min_samples_split': 19, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.6936605538806662.
[I 2026-01-17 11:05:43,850] Trial 1 finished with value: 0.739064710631262 and parameters: {'max_depth': 25, 'min_samples_split': 17, 'min_samples_leaf': 2}. Best is trial 1 with value: 0.739064710631262.
[I 2026-01-17 11:05:44,934] Trial 2 finished with value: 0.645574179731276 and parameters: {'max_depth': 9, 'min_samples_split': 12, 'min_samples_leaf': 1}. Best is trial 1 with value: 0.739064710631262.
[I 2026-01-17 11:05:45,668] Trial 3 finished with value: 0.356427856620105 and parameters: {'max_depth': 3, 'min_samples_split': 19, 'min_samples_leaf': 6}. Best is trial 1 with value: 0.739064710631262.
[I 2026-01-17 11:05:46,946] Trial 4 finished with value: 0.7164992091690978 and parameters: {'max_depth': 14, 'min_samples_split': 5, 'min_samples_leaf': 7}. Best is trial 1 with v

...

[I 2026-01-17 11:17:24,858] Trial 0 finished with value: 0.7606890466558739 and parameters: {'n_estimators': 202, 'max_samples': 0.7515379893026695, 'max_features': 0.6021710088428585}. Best is trial 0 with value: 0.7606890466558739.
[I 2026-01-17 11:19:19,968] Trial 1 finished with value: 0.7550448811346854 and parameters: {'n_estimators': 333, 'max_samples': 0.75088009565467, 'max_features': 0.5743405689739394}. Best is trial 0 with value: 0.7606890466558739.
[I 2026-01-17 11:22:00,592] Trial 2 finished with value: 0.7712048451485611 and parameters: {'n_estimators': 295, 'max_samples': 0.7795760272394714, 'max_features': 0.8927569321734097}. Best is trial 2 with value: 0.7712048451485611.
[I 2026-01-17 11:25:27,275] Trial 3 finished with value: 0.7715542618058419 and parameters: {'n_estimators': 376, 'max_samples': 0.8031005971973714, 'max_features': 0.8954462467161454}. Best is trial 3 with value: 0.7715542618058419.
[I 2026-01-17 11:28:09,634] Trial 4 finished with value: 0.7589705


📊 RESULTS FOR MODEL: Bagging
🎯🎯 TARGET: TYPE
-------------------------------------------------------
📈 Accuracy        : 0.94
🧠 Precision (mac) : 0.93
🔁 Recall    (mac) : 0.93
🏆 F1-score  (mac) : 0.93
📊 K-Fold F1 Mean  : 0.95
📉 K-Fold F1 Std   : 0.00

-------------------------------------------------------
🎯 TARGET: CVSS_SCORE
-------------------------------------------------------
📈 Accuracy        : 0.61
🧠 Precision (mac) : 0.62
🔁 Recall    (mac) : 0.41
🏆 F1-score  (mac) : 0.43
📊 K-Fold F1 Mean  : 0.64
📉 K-Fold F1 Std   : 0.00

📌 MEAN F1 (TYPE + CVSS) / 2 : 0.68

🔥 NOW TUNING: Bagged DT


[I 2026-01-17 13:04:43,639] Trial 0 finished with value: 0.7716735680221729 and parameters: {'n_estimators': 228}. Best is trial 0 with value: 0.7716735680221729.
[I 2026-01-17 13:05:55,124] Trial 1 finished with value: 0.7709765595469276 and parameters: {'n_estimators': 86}. Best is trial 0 with value: 0.7716735680221729.
[I 2026-01-17 13:07:10,287] Trial 2 finished with value: 0.7713206623722857 and parameters: {'n_estimators': 81}. Best is trial 0 with value: 0.7716735680221729.
[I 2026-01-17 13:12:18,452] Trial 3 finished with value: 0.771593061918053 and parameters: {'n_estimators': 324}. Best is trial 0 with value: 0.7716735680221729.
[I 2026-01-17 13:14:54,848] Trial 4 finished with value: 0.771539818041038 and parameters: {'n_estimators': 188}. Best is trial 0 with value: 0.7716735680221729.
[I 2026-01-17 13:20:21,110] Trial 5 finished with value: 0.7719989681566057 and parameters: {'n_estimators': 377}. Best is trial 5 with value: 0.7719989681566057.
[I 2026-01-17 13:26:27,236


📊 RESULTS FOR MODEL: Bagged DT
🎯🎯 TARGET: TYPE
-------------------------------------------------------
📈 Accuracy        : 0.93
🧠 Precision (mac) : 0.92
🔁 Recall    (mac) : 0.92
🏆 F1-score  (mac) : 0.92
📊 K-Fold F1 Mean  : 0.95
📉 K-Fold F1 Std   : 0.00

-------------------------------------------------------
🎯 TARGET: CVSS_SCORE
-------------------------------------------------------
📈 Accuracy        : 0.59
🧠 Precision (mac) : 0.57
🔁 Recall    (mac) : 0.41
🏆 F1-score  (mac) : 0.42
📊 K-Fold F1 Mean  : 0.64
📉 K-Fold F1 Std   : 0.00

📌 MEAN F1 (TYPE + CVSS) / 2 : 0.67

🔥 NOW TUNING: GradientBoosting


[I 2026-01-17 15:19:07,886] Trial 0 finished with value: 0.716202703475722 and parameters: {'n_estimators': 248, 'learning_rate': 0.013608700544063471, 'max_depth': 4}. Best is trial 0 with value: 0.716202703475722.
[I 2026-01-17 15:31:26,379] Trial 1 finished with value: 0.7685906384053242 and parameters: {'n_estimators': 198, 'learning_rate': 0.12167376249832161, 'max_depth': 6}. Best is trial 1 with value: 0.7685906384053242.


In [ ]:
console = Console()

result_sorted = sorted(results, key=lambda i: i[3], reverse=True)

best_model = max(results, key=lambda x: x[3])
worst_model = min(results, key=lambda x: x[3])

table = Table(title="Optuna", show_lines=True)
table.add_column("Algorithm")
table.add_column("Accuracy")
table.add_column("Precision")
table.add_column("Recall")
table.add_column("F1-score")
table.add_column("K-Fold mean")
table.add_column("K-Fold std")

for row in result_sorted:
    algo, acc, presicion, recall, f1, kmean, kstd = row

    if row == best_model:
        table.add_row(
            f"[bold green]{algo}[/bold green]",
            f"[bold green]{acc:.2f}[/bold green]",
            f"[bold green]{presicion:.2f}[/bold green]",
            f"[bold green]{recall:.2f}[/bold green]",
            f"[bold green]{f1:.2f}[/bold green]",
            f"[bold green]{kmean:.2f}[/bold green]",
            f"[bold green]{kstd:.2f}[/bold green]",
        )
    elif row == worst_model:
        table.add_row(
            f"[bold red]{algo}[/bold red]",
            f"[bold red]{acc:.2f}[/bold red]",
            f"[bold red]{presicion:.2f}[/bold red]",
            f"[bold red]{recall:.2f}[/bold red]",
            f"[bold red]{f1:.2f}[/bold red]",
            f"[bold red]{kmean:.2f}[/bold red]",
            f"[bold red]{kstd:.2f}[/bold red]",
        )
    else:
        table.add_row(algo, f"{acc:.2f}", f"{presicion:.2f}", f"{recall:.2f}", f"{f1:.2f}", f"{kmean:.2f}", f"{kstd:.2f}")

console.print(table)

                                            Optuna                                            
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Algorithm            ┃ Accuracy ┃ Precision ┃ Recall ┃ F1-score ┃ K-Fold mean ┃ K-Fold std ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ SGDC                 │ 0.38     │ 0.38      │ 1.00   │ 0.55     │ 0.33        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Stacking             │ 0.89     │ 0.86      │ 0.83   │ 0.85     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Bagged DT            │ 0.88     │ 0.85      │ 0.83   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Bagging              │ 0.89     │ 0.86      │ 0.83   │ 0.85     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ LGBMClassifier       │ 0.88     │ 0.86      │ 0.83   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ GradientBoosting     │ 0.88     │ 0.85      │ 0.82   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ HistGradientBoosting │ 0.87     │ 0.84      │ 0.82   │ 0.83     │ 0.89        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Soft Voting          │ 0.88     │ 0.86      │ 0.82   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ CatBoost             │ 0.88     │ 0.86      │ 0.82   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ XGBoost              │ 0.88     │ 0.86      │ 0.82   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Hard Voting          │ 0.88     │ 0.85      │ 0.82   │ 0.83     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ RandomForest         │ 0.88     │ 0.87      │ 0.82   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ ExtraTrees           │ 0.87     │ 0.84      │ 0.81   │ 0.83     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ SVC                  │ 0.84     │ 0.77      │ 0.81   │ 0.79     │ 0.84        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ DecisionTree         │ 0.85     │ 0.81      │ 0.80   │ 0.80     │ 0.86        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ AdaBoost             │ 0.83     │ 0.76      │ 0.80   │ 0.78     │ 0.83        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Passive Aggressive   │ 0.76     │ 0.65      │ 0.79   │ 0.71     │ 0.73        │ 0.02       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ KNN                  │ 0.78     │ 0.68      │ 0.79   │ 0.73     │ 0.83        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ MLPC                 │ 0.84     │ 0.78      │ 0.78   │ 0.78     │ 0.85        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Logistic

In [ ]:
temp_console = Console(record=True)
temp_console.print(table)
text = temp_console.export_text()
with open('results/Tuning.txt', 'a', encoding='utf-8') as f:
    f.write(text)

                                            Optuna                                            
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Algorithm            ┃ Accuracy ┃ Precision ┃ Recall ┃ F1-score ┃ K-Fold mean ┃ K-Fold std ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ SGDC                 │ 0.38     │ 0.38      │ 1.00   │ 0.55     │ 0.33        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Stacking             │ 0.89     │ 0.86      │ 0.83   │ 0.85     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Bagged DT            │ 0.88     │ 0.85      │ 0.83   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Bagging              │ 0.89     │ 0.86      │ 0.83   │ 0.85     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ LGBMClassifier       │ 0.88     │ 0.86      │ 0.83   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ GradientBoosting     │ 0.88     │ 0.85      │ 0.82   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ HistGradientBoosting │ 0.87     │ 0.84      │ 0.82   │ 0.83     │ 0.89        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Soft Voting          │ 0.88     │ 0.86      │ 0.82   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ CatBoost             │ 0.88     │ 0.86      │ 0.82   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ XGBoost              │ 0.88     │ 0.86      │ 0.82   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Hard Voting          │ 0.88     │ 0.85      │ 0.82   │ 0.83     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ RandomForest         │ 0.88     │ 0.87      │ 0.82   │ 0.84     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ ExtraTrees           │ 0.87     │ 0.84      │ 0.81   │ 0.83     │ 0.90        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ SVC                  │ 0.84     │ 0.77      │ 0.81   │ 0.79     │ 0.84        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ DecisionTree         │ 0.85     │ 0.81      │ 0.80   │ 0.80     │ 0.86        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ AdaBoost             │ 0.83     │ 0.76      │ 0.80   │ 0.78     │ 0.83        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Passive Aggressive   │ 0.76     │ 0.65      │ 0.79   │ 0.71     │ 0.73        │ 0.02       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ KNN                  │ 0.78     │ 0.68      │ 0.79   │ 0.73     │ 0.83        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ MLPC                 │ 0.84     │ 0.78      │ 0.78   │ 0.78     │ 0.85        │ 0.00       │
├──────────────────────┼──────────┼───────────┼────────┼──────────┼─────────────┼────────────┤
│ Logistic

# Another Tunings

<a href="../Tuning/manual_search.ipynb" target="_blank" style="text-decoration: none;">
    <button style="
        background-color: #45a049;
        color: white; 
        padding: 12px 24px; 
        font-size: 16px; 
        border: none; 
        border-radius: 8px;
        cursor: pointer;
        transition: background-color 0.3s;
    " onmouseover="this.style.backgroundColor='#45a049'" onmouseout="this.style.backgroundColor='#4CAF50'">
        Manual Search
    </button>
</a>

<a href="../Tuning/optuna.ipynb" target="_blank" style="text-decoration: none;">
    <button style="
        background-color: #45a049;
        color: white; 
        padding: 12px 24px; 
        font-size: 16px; 
        border: none; 
        border-radius: 8px;
        cursor: pointer;
        transition: background-color 0.3s;
    " onmouseover="this.style.backgroundColor='#45a049'" onmouseout="this.style.backgroundColor='#4CAF50'">
        optuna
    </button>
</a>

<a href="../Tuning/random_search.ipynb" target="_blank" style="text-decoration: none;">
    <button style="
        background-color: #45a049;
        color: white; 
        padding: 12px 24px; 
        font-size: 16px; 
        border: none; 
        border-radius: 8px;
        cursor: pointer;
        transition: background-color 0.3s;
    " onmouseover="this.style.backgroundColor='#45a049'" onmouseout="this.style.backgroundColor='#4CAF50'">
        Random Search
    </button>
</a>